In [2]:
import pandas as pd
import numpy as np

In [4]:
speed = pd.read_csv("./datasets/windGuajira2019.csv", skiprows= 3)
speed.head()

,time,local_time,electricity,wind_speed
0,2019-01-01 00:00,2018-12-31 19:00,0.952,13.742
1,2019-01-01 01:00,2018-12-31 20:00,0.953,13.783
2,2019-01-01 02:00,2018-12-31 21:00,0.946,13.515
3,2019-01-01 03:00,2018-12-31 22:00,0.940,13.327
4,2019-01-01 04:00,2018-12-31 23:00,0.941,13.341


In [5]:
speed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   time         8760 non-null   object 
 1   local_time   8760 non-null   object 
 2   electricity  8760 non-null   float64
 3   wind_speed   8760 non-null   float64
dtypes: float64(2), object(2)
memory usage: 273.9+ KB
